# 04 — Fatores Socioeconômicos (ENEM 2023)

Análises sem rótulos (apenas colunas cruas):
- **Escolaridade dos pais** (`Q001`, `Q002`) × renda (`Q006_ord`) × desempenho (`NOTA_MEDIA_5`).
- **Tipo de escola** (`TP_ESCOLA`) × nota média.
- **Matriz de correlação cruzada** entre renda e escolaridade vs notas.

**Requisitos:** `../data/interim/enem_2023.parquet` com as colunas: `Q001`, `Q002`, `Q006`, `TP_ESCOLA`, `NOTA_MEDIA_5` e as notas por área.  
Obs.: Se `Q001_ord`, `Q002_ord`, `Q006_ord` não existirem, este notebook cria automaticamente a ordem A=1, B=2, ...

_Somente matplotlib; não salva figuras em disco._

In [ ]:
%matplotlib inline
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DATA_PATH = Path("../data/interim/enem_2023.parquet")
print("Lendo:", DATA_PATH)
df = pd.read_parquet(DATA_PATH)
print("Dimensões:", df.shape)

REQUIRED = [
    "Q001", "Q002", "Q006", "TP_ESCOLA",
    "NU_NOTA_MT", "NU_NOTA_LC", "NU_NOTA_CH", "NU_NOTA_CN", "NU_NOTA_REDACAO",
    "NOTA_MEDIA_5"
]
missing = [c for c in REQUIRED if c not in df.columns]
if missing:
    raise RuntimeError(f"Faltam colunas necessárias: {missing}")

# filtra treineiros se houver
if "IN_TREINEIRO" in df.columns:
    df = df[df["IN_TREINEIRO"] == 0].copy()

# garante ordinais: A=1, B=2, ..., caso *_ord não existam
def ensure_ord(df, code_col, ord_col):
    if ord_col in df.columns:
        return df
    # mapeia letras A..Z -> 1..26 (mantém NaN para códigos fora desse padrão)
    letters = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
    mapping = {ch: i+1 for i, ch in enumerate(letters)}
    df[ord_col] = df[code_col].map(mapping)
    return df

df = ensure_ord(df, "Q001", "Q001_ord")
df = ensure_ord(df, "Q002", "Q002_ord")
df = ensure_ord(df, "Q006", "Q006_ord")

# dropa linhas sem info mínima
df = df.dropna(subset=["Q001_ord", "Q002_ord", "Q006_ord", "NOTA_MEDIA_5"]).copy()
df.head(3)

## 1) Escolaridade dos Pais × Nota Média
Usamos a ordem ordinal (A=1, B=2, ...) para representar crescente escolaridade declarada.

In [ ]:
# Q001 (Escolaridade do responsável 1) vs Média
g1 = df.groupby("Q001_ord", as_index=False)["NOTA_MEDIA_5"].mean().sort_values("Q001_ord")
plt.figure(figsize=(9, 4.8))
plt.plot(g1["Q001_ord"], g1["NOTA_MEDIA_5"], marker="o")
plt.title("Tendência: Escolaridade Q001 (ord) × Nota Média")
plt.xlabel("Q001_ord (A=1, B=2, ...)")
plt.ylabel("Nota média (5 provas)")
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

# Q002 (Escolaridade do responsável 2) vs Média
g2 = df.groupby("Q002_ord", as_index=False)["NOTA_MEDIA_5"].mean().sort_values("Q002_ord")
plt.figure(figsize=(9, 4.8))
plt.plot(g2["Q002_ord"], g2["NOTA_MEDIA_5"], marker="o")
plt.title("Tendência: Escolaridade Q002 (ord) × Nota Média")
plt.xlabel("Q002_ord (A=1, B=2, ...)")
plt.ylabel("Nota média (5 provas)")
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

# Correlações simples
print("Correlação Q001_ord × NOTA_MEDIA_5:", round(df["Q001_ord"].corr(df["NOTA_MEDIA_5"]), 3))
print("Correlação Q002_ord × NOTA_MEDIA_5:", round(df["Q002_ord"].corr(df["NOTA_MEDIA_5"]), 3))

## 2) Escolaridade dos Pais × Renda (consistência socioeconômica)
Tendências esperadas: maior escolaridade tende a coexistir com maior renda (`Q006_ord`).

In [ ]:
g1r = df.groupby("Q001_ord", as_index=False)["Q006_ord"].mean().sort_values("Q001_ord")
plt.figure(figsize=(9, 4.8))
plt.plot(g1r["Q001_ord"], g1r["Q006_ord"], marker="o")
plt.title("Tendência: Escolaridade Q001 (ord) × Renda (Q006_ord)")
plt.xlabel("Q001_ord")
plt.ylabel("Q006_ord (A=1, B=2, ...)")
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

g2r = df.groupby("Q002_ord", as_index=False)["Q006_ord"].mean().sort_values("Q002_ord")
plt.figure(figsize=(9, 4.8))
plt.plot(g2r["Q002_ord"], g2r["Q006_ord"], marker="o")
plt.title("Tendência: Escolaridade Q002 (ord) × Renda (Q006_ord)")
plt.xlabel("Q002_ord")
plt.ylabel("Q006_ord (A=1, B=2, ...)")
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

print("Correlação Q001_ord × Q006_ord:", round(df["Q001_ord"].corr(df["Q006_ord"]), 3))
print("Correlação Q002_ord × Q006_ord:", round(df["Q002_ord"].corr(df["Q006_ord"]), 3))

## 3) Tipo de Escola × Nota Média
Se `TP_ESCOLA_rot` não existir, usamos `TP_ESCOLA` com rótulos padrão: 1=Pública, 2=Privada, 3=Exterior (valores não mapeáveis permanecem como código).

In [ ]:
if "TP_ESCOLA_rot" in df.columns:
    grp = df.groupby("TP_ESCOLA_rot", as_index=False)["NOTA_MEDIA_5"].mean()
    x = grp["TP_ESCOLA_rot"].tolist()
    y = grp["NOTA_MEDIA_5"].tolist()
else:
    # mapeamento seguro (ajuste se necessário)
    map_escola = {1: "Pública", 2: "Privada", 3: "Exterior"}
    tmp = df[["TP_ESCOLA", "NOTA_MEDIA_5"]].copy()
    tmp["TP_ESCOLA_lab"] = tmp["TP_ESCOLA"].map(map_escola).fillna(tmp["TP_ESCOLA"].astype(str))
    grp = tmp.groupby("TP_ESCOLA_lab", as_index=False)["NOTA_MEDIA_5"].mean()
    x = grp["TP_ESCOLA_lab"].tolist()
    y = grp["NOTA_MEDIA_5"].tolist()

plt.figure(figsize=(7.5, 4.8))
plt.bar(x, y)
plt.title("Nota Média por Tipo de Escola")
plt.xlabel("Tipo de Escola")
plt.ylabel("Nota média (5 provas)")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

grp.sort_values("NOTA_MEDIA_5", ascending=False)

## 4) Matriz de Correlação Cruzada
Correlação de Pearson entre **renda (Q006_ord)**, **escolaridade (Q001_ord, Q002_ord)** e **desempenho** (média e áreas).  
Visualização com `matplotlib.imshow` (sem definir cores explicitamente).

In [ ]:
cols_corr = [
    "Q006_ord", "Q001_ord", "Q002_ord",
    "NOTA_MEDIA_5", "NU_NOTA_MT", "NU_NOTA_LC", "NU_NOTA_CH", "NU_NOTA_CN", "NU_NOTA_REDACAO"
]
sub = df[cols_corr].dropna()
corr = sub.corr(numeric_only=True)
print(corr.round(3))

plt.figure(figsize=(7, 6))
plt.imshow(corr, aspect='auto')
plt.title("Correlação — Renda, Escolaridade e Notas")
plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha='right')
plt.yticks(range(len(corr.index)), corr.index)
plt.tight_layout()
plt.show()

## 5) Observações para apresentação
- **Renda e escolaridade** dos pais apresentam correlação positiva entre si e com a **nota média**.
- A **diferença por tipo de escola** é visível na média, mas não explica tudo — renda e escolaridade seguem relevantes.
- Entre as áreas, costuma-se observar correlação mais forte em **Matemática** e **Redação** (confirmar nos seus números).